In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Adjust this if your path differs
csv_path = "/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv"

df = pd.read_csv(csv_path)
print("Columns in CSV:\n", df.columns.tolist())


Columns in CSV:
 ['filename', 'date', 'datetime', 'air_temp_C', 'dew_point_C', 'relative_humidity_percent', 'wind_speed_m_s', 'precipitation_in']


In [ ]:
# Exact meteorological columns from your CSV:
weather_cols = [
    "air_temp_C",
    "dew_point_C",
    "relative_humidity_percent",
    "wind_speed_m_s",
    "precipitation_in"
]

# Load and clean
df = pd.read_csv(csv_path)
for col in weather_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=weather_cols).reset_index(drop=True)

# Build TensorDataset
weather_tensor = torch.tensor(df[weather_cols].values, dtype=torch.float32)
dataset        = TensorDataset(weather_tensor)
loader         = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
Z_DIM  = 16                     # size of latent noise
DIMS   = len(weather_cols)      # number of features = 5
HIDDEN = 64                     # hidden layer size

class WeatherGenerator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(Z_DIM,  HIDDEN),
            nn.ReLU(True),
            nn.Linear(HIDDEN, HIDDEN),
            nn.ReLU(True),
            nn.Linear(HIDDEN, DIMS),
        )
    def forward(self, z):
        return self.net(z)

class WeatherDiscriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(DIMS,   HIDDEN),
            nn.LeakyReLU(0.2, True),
            nn.Linear(HIDDEN, HIDDEN),
            nn.LeakyReLU(0.2, True),
            nn.Linear(HIDDEN, 1),
        )
    def forward(self, x):
        return self.net(x).view(-1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = WeatherGenerator().to(device)
D = WeatherDiscriminator().to(device)


In [ ]:
opt_G = optim.Adam(G.parameters(), lr=2e-4, betas=(0.5,0.999))
opt_D = optim.Adam(D.parameters(), lr=2e-4, betas=(0.5,0.999))
bce   = nn.BCEWithLogitsLoss()


In [ ]:
EPOCHS = 250

for epoch in range(1, EPOCHS+1):
    for (real_vec,) in loader:
        real_vec = real_vec.to(device)
        bs       = real_vec.size(0)

        # — Discriminator step —
        noise    = torch.randn(bs, Z_DIM, device=device)
        fake_vec = G(noise).detach()
        d_real   = D(real_vec)
        d_fake   = D(fake_vec)
        d_loss   = 0.5*(bce(d_real, torch.ones(bs,device=device)) +
                       bce(d_fake, torch.zeros(bs,device=device)))
        opt_D.zero_grad(); d_loss.backward(); opt_D.step()

        # — Generator step —
        noise   = torch.randn(bs, Z_DIM, device=device)
        fake    = G(noise)
        g_loss  = bce(D(fake), torch.ones(bs,device=device))
        opt_G.zero_grad(); g_loss.backward(); opt_G.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | D_loss: {d_loss:.4f} | G_loss: {g_loss:.4f}")


Epoch 010 | D_loss: 0.6676 | G_loss: 0.7477
Epoch 020 | D_loss: 0.6246 | G_loss: 0.7290
Epoch 030 | D_loss: 0.7021 | G_loss: 0.6133
Epoch 040 | D_loss: 0.7037 | G_loss: 0.7800
Epoch 050 | D_loss: 0.6826 | G_loss: 0.8460
Epoch 060 | D_loss: 0.6576 | G_loss: 0.6334
Epoch 070 | D_loss: 0.6749 | G_loss: 0.8861
Epoch 080 | D_loss: 0.6473 | G_loss: 0.9656
Epoch 090 | D_loss: 0.6747 | G_loss: 0.7491
Epoch 100 | D_loss: 0.6949 | G_loss: 0.6976
Epoch 110 | D_loss: 0.7235 | G_loss: 0.5369
Epoch 120 | D_loss: 0.7030 | G_loss: 0.9899
Epoch 130 | D_loss: 0.6331 | G_loss: 0.7676
Epoch 140 | D_loss: 0.6439 | G_loss: 0.6441
Epoch 150 | D_loss: 0.6584 | G_loss: 0.5584
Epoch 160 | D_loss: 0.6753 | G_loss: 0.8507
Epoch 170 | D_loss: 0.6442 | G_loss: 0.8793
Epoch 180 | D_loss: 0.6398 | G_loss: 0.7341
Epoch 190 | D_loss: 0.6234 | G_loss: 0.5519
Epoch 200 | D_loss: 0.6322 | G_loss: 0.8164
Epoch 210 | D_loss: 0.6812 | G_loss: 0.5948
Epoch 220 | D_loss: 0.6521 | G_loss: 0.7549
Epoch 230 | D_loss: 0.6151 | G_l

In [ ]:
torch.save(G.state_dict(), "/content/drive/MyDrive/weather_generator.pth")
torch.save(D.state_dict(), "/content/drive/MyDrive/weather_discriminator.pth")


In [ ]:
# === Cell N+1: plot & save GAN loss curves ===
import matplotlib.pyplot as plt

# Convert any torch tensors to floats
d_vals = [dl.cpu().item() if hasattr(dl, 'cpu') else float(dl) for dl in d_loss]
g_vals = [gl.cpu().item() if hasattr(gl, 'cpu') else float(gl) for gl in g_loss]

# x-axis ticks every 10 epochs
epochs = list(range(10, EPOCHS + 1, 10))

plt.figure(figsize=(8,5))
plt.plot(epochs, d_vals, label='Discriminator Loss')
plt.plot(epochs, g_vals, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN Loss Curves')
plt.legend(loc='best')

# Save to PNG
plt.savefig('/content/drive/MyDrive/gan_loss_curves.png', dpi=300, bbox_inches='tight')
plt.show()


TypeError: iteration over a 0-d tensor